In [23]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader,
    PyPDFLoader,
    Docx2txtLoader
)

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()


True

In [ ]:
doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx",
]

docs = []
for doc_file in doc_paths:
    file_path = Path(doc_file)
    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(str(file_path))
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(str(file_path))
        elif doc_file.endswith(".txt"):
            loader = TextLoader(str(file_path))
        else:
            continue
        docs.extend(loader.load())
    except Exception as e:
        print(f"Error loading {doc_file}: {e}")

    finally:
        os.remove(file_path)    


# Load URLs
url = "https://docs.streamlit.io/develop/quick-reference/release-notes"      
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())  
except Exception as e:
    print(f"Error loading {url}: {e}")

In [12]:
docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-15T19:40:36+02:00', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_siteid': 'eb25818e-5bd5-49bf-99de-53e3e7b42630', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_method': 'Standard', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_enabled': 'True', 'author': 'Domingo Domènech Enric (ERNI)', 'moddate': '2024-09-15T19:40:36+02:00', 'source': 'docs/test_rag.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='My favorite food is margarita pizza. \nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'docs/test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog of highlights and fixes for th

In [13]:
# Split Docs
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)


document_chunks = text_splitter.split_documents(docs)

In [15]:
# Tokenize and load the documents to the vector store
# print(f"Tokenizing and loading documents to vector store...{os.getenv("OPENAI_API_KEY")}")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=embeddings,
)

/Users/raihanarman/python_project/Chat App-LLM-RAG/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [28]:
def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation, focusing on the most recent messages.")
    ])

    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)
    
    return retriever_chain

In [29]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])

    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [30]:
llm_stream = Ollama(
    model="deepseek-r1:8b",
    temperature=0.3,
)

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]

messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


The latest version of Streamlit is **1.53.0**, released on **January 14, 2026**.

To upgrade to this version, run:

```bash
pip install --upgrade streamlit
```